<a href="https://colab.research.google.com/github/Vijayarajan-S/Fish-Classification-using-CNN/blob/main/cnnfishclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [ ]:
!pip install gdown

# Use gdown with the folder link
!gdown --folder https://drive.google.com/drive/folders/1iKdOs4slf3XvNWkeSfsszhPRggfJ2qEd -O /content/Dataset


Retrieving folder contents
Processing file 1GHqB7F1AYocMbiUcqOa4VFtw5mJH0wmb Dataset.zip
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1GHqB7F1AYocMbiUcqOa4VFtw5mJH0wmb
From (redirected): https://drive.google.com/uc?id=1GHqB7F1AYocMbiUcqOa4VFtw5mJH0wmb&confirm=t&uuid=6627996b-20a1-45fa-99bf-e4989944c71c
To: /content/Dataset/Dataset.zip
100% 271M/271M [00:03<00:00, 82.9MB/s]
Download completed


In [ ]:
!unzip '/content/Dataset/Dataset.zip'

Archive:  /content/Dataset/Dataset.zip
replace images.cv_jzk6llhf18tm3k0kyttxz/readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import tensorflow as tf

# Set paths
train_dir = "/content/images.cv_jzk6llhf18tm3k0kyttxz/data/train"
val_dir = "/content/images.cv_jzk6llhf18tm3k0kyttxz/data/val"
test_dir = "/content/images.cv_jzk6llhf18tm3k0kyttxz/data/test"

# Load datasets
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),  # resize all images to 224x224
    batch_size=16,
    shuffle=True
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(224, 224),
    batch_size=16,
    shuffle=False
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=16,
    shuffle=False
)


In [ ]:
num_classes = 11
model=Sequential()

# Block 1
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block 2
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block 3
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Block 4
model = Sequential()
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))



In [ ]:
class_names = train_dataset.class_names
print(class_names)


In [ ]:



model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [ ]:
# Get one batch from your training dataset
x_batch, y_batch = next(train_dataset.as_numpy_iterator())

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i+1)
    plt.imshow(x_batch[i].astype("uint8"))
    plt.title(f"Label: {y_batch[i]}")
    plt.axis("off")
plt.show()


In [ ]:
import datetime
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint

# TensorBoard log
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True
)

# Save best model
checkpoint_callback = ModelCheckpoint(
    "best_model.keras",
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)

# Train model
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=10,
    callbacks=[tensorboard_callback, checkpoint_callback]
)


Epoch 1/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.5703 - loss: 1.4480
Epoch 1: val_accuracy improved from -inf to 0.60073, saving model to best_model.keras
390/390 ━━━━━━━━━━━━━━━━━━━━ 1277s 3s/step - accuracy: 0.5706 - loss: 1.4468 - val_accuracy: 0.6007 - val_loss: 1.4274
Epoch 2/10
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.8503 - loss: 0.4520
Epoch 2: val_accuracy improved from 0.60073 to 0.78480, saving model to best_model.keras
390/390 ━━━━━━━━━━━━━━━━━━━━ 1257s 3s/step - accuracy: 0.8503 - loss: 0.4520 - val_accuracy: 0.7848 - val_loss: 0.6144
Epoch 3/10
339/390 ━━━━━━━━━━━━━━━━━━━━ 2:36 3s/step - accuracy: 0.8912 - loss: 0.3143

In [ ]:
predictions = model.predict(test_dataset)
print(predictions.shape)

In [ ]:
import matplotlib.pyplot as plt

x_batch, y_batch = next(iter(test_dataset))
batch_predictions = model.predict(x_batch)
batch_classes = np.argmax(batch_predictions, axis=1)

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(x_batch[i].numpy().astype("uint8"))
    plt.title(f"Pred: {batch_classes[i]}, True: {y_batch[i].numpy()}")
    plt.axis("off")
plt.show()
